In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import matplotlib.pyplot as plt

from preprocessing import split_X_y
from model_optimization import run_optimization_all_classifiers

import warnings
warnings.filterwarnings('ignore')

In [2]:
DATA_DIR = os.path.join(os.getcwd(), 'data')

## Import Labeled Data

In [3]:
data = pd.read_csv(os.path.join(DATA_DIR, 'ind4_clustering_10PCs.csv'))
data

,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,cluster_label
PC_1,,,,,,,,,,
-18.265232,0.179890,2.512753,-0.017214,-1.253427,5.103237,3.069733,-1.498417,-0.213276,-1.921352,B
-11.648572,-2.388439,0.988511,3.860645,1.748230,0.217374,-0.771974,0.272593,1.387455,-0.643988,B
2.597364,-1.249371,-3.363064,-2.655913,-1.898509,-0.066221,-3.849705,2.200412,-2.473733,-0.130113,L1
-3.417240,-4.438395,0.636533,3.351176,0.887752,-7.494596,-0.488962,0.585061,2.802112,-1.061658,B
7.382528,-0.940683,-9.480106,-6.126398,2.831281,1.671729,0.735573,-5.483888,3.156084,-4.968595,L1
...,...,...,...,...,...,...,...,...,...,...
8.322987,-3.214379,-0.689009,-0.262546,1.024876,-2.575015,-1.277584,2.146089,1.676037,0.391478,L1
-11.008588,-1.900832,-2.101015,2.335349,1.157123,-4.229745,0.003551,1.722289,-0.468227,2.221812,B
9.320722,-5.689082,2.899196,2.080614,-1.207569,-1.014811,5.033344,2.736327,-3.361486,1.148018,L1


In [4]:
data['cluster_label'].value_counts()

cluster_label
L1              1931
B               1429
L2               572
Unclassified      82
Name: count, dtype: int64

In [5]:
data.describe()

,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10
count,4014.000000,4014.000000,4014.000000,4014.000000,4014.000000,4014.000000,4014.000000,4014.000000,4014.000000
mean,-0.021141,-0.078401,0.083243,0.065213,0.008302,0.070433,-0.011541,-0.050991,-0.019961
std,6.167741,4.481838,4.296681,3.434765,3.133784,2.905260,2.792151,2.708235,2.444898
min,-9.499155,-13.907784,-43.566033,-57.245970,-14.597362,-21.256365,-15.068132,-8.321540,-8.657476
25%,-3.430494,-2.636772,-1.408544,-0.966391,-1.665316,-1.810213,-1.357253,-1.739949,-1.497361
50%,-1.893019,-0.082368,0.591807,0.255896,0.038975,0.065079,0.198538,-0.242970,-0.049414
75%,-0.071587,2.207989,2.314132,1.397715,1.754820,2.037075,1.666112,1.390688,1.504905
max,22.768726,34.417159,10.826321,10.860815,11.885671,8.529058,11.644395,20.034509,9.515228


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4014 entries, -18.2652321254284 to -10.4338701401273
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PC_2           4014 non-null   float64
 1   PC_3           4014 non-null   float64
 2   PC_4           4014 non-null   float64
 3   PC_5           4014 non-null   float64
 4   PC_6           4014 non-null   float64
 5   PC_7           4014 non-null   float64
 6   PC_8           4014 non-null   float64
 7   PC_9           4014 non-null   float64
 8   PC_10          4014 non-null   float64
 9   cluster_label  4014 non-null   object 
dtypes: float64(9), object(1)
memory usage: 345.0+ KB


## PreProcessing

Split X, y and turn labels from string to int with values in range (0, n_classes-1).

In [7]:
X, y = split_X_y(data)

['PC_2', 'PC_3', 'PC_4', 'PC_5', 'PC_6', 'PC_7', 'PC_8', 'PC_9', 'PC_10']


## Optimize each model and compare them

In [ ]:
estimators_dict, best_params_dict = run_optimization_all_classifiers(X, y)
best_params_dict

[I 2023-06-22 11:59:00,182] A new study created in memory with name: no-name-1408ab26-304f-4d9a-aec1-ef141314ae8b
/home/daphne/.pyenv/versions/3.9.1/envs/machineLearningProjects/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/daphne/.pyenv/versions/3.9.1/envs/machineLearningProjects/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/daphne/.pyenv/versions/3.9.1/envs/machineLearningProjects/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/daphne/.pyenv/versions/3.9.1/envs/machineLearningProjects/lib/python3.9/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/h

Starting optimization for classifier svm


  0%|          | 0/50 [00:00<?, ?it/s]